## Graph U-Net Training Pipeline


###  Drive Mount & Installation

We mount Google Drive to access the data and install the required **PyTorch Geometric** libraries for our EGNN implementation.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
!pip install torch-geometric

We install the library specific for EGNN implementation

In [ ]:
# Librairie spécifique pour ton modèle EGNN
!pip install egnn-pytorch

# Config

In [ ]:
%%writefile config.py
"""Configuration and hyperparameters of the model"""

import torch


class Config:
    # ------------------------------------------------------------------
    # Data (absolute paths)
    # ------------------------------------------------------------------
    TRAIN_DATA_PATH = (
        "/content/gdrive/MyDrive/DataSetML4/data/train_data.pt"
    )
    VAL_DATA_PATH = (
        "/content/gdrive/MyDrive/DataSetML4/data/val_data.ptt"
    )

    TRAIN_SUBSET_RATIO = 1
    BATCH_SIZE = 2

    # ------------------------------------------------------------------
    # Architecture
    # ------------------------------------------------------------------
    IN_NODE_FEATURES = 9
    HIDDEN_CHANNELS = 32
    OUT_STRESS_DIM = 1
    DEPTH = 4

    LAMBDA_STRESS = 0.5

    # EGNN physics
    NUM_NEIGHBORS = 16
    UPDATE_COORS = True
    UPDATE_FEATS = True

    # ------------------------------------------------------------------
    # Training
    # ------------------------------------------------------------------
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.0005
    WEIGHT_DECAY = 1e-16
    GRADIENT_CLIP = 1.0

    # ------------------------------------------------------------------
    # Saving
    # ------------------------------------------------------------------
    BEST_MODEL_PATH = "best_model.pt"

    TRAINING_CURVE_PATH = (
        f"training_curve_R{TRAIN_SUBSET_RATIO}_"
        f"B{BATCH_SIZE}_"
        f"H{HIDDEN_CHANNELS}_"
        f"D{DEPTH}_"
        f"E{NUM_EPOCHS}.png"
    )

    # ------------------------------------------------------------------
    # Device
    # ------------------------------------------------------------------
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Utils

In [ ]:
%%writefile utils.py
"""Utils : normalizer and helpers."""
import torch

class UnitGaussianNormalizer:
    """Gaussian Normalizer for the data"""

    def __init__(self, x=None, eps=1e-5):
        self.eps = eps
        if x is not None:
            self.mean = x.mean(dim=0, keepdim=True)
            self.std = x.std(dim=0, keepdim=True) + eps
        else:
            self.mean = None
            self.std = None

    def encode(self, x):
        if self.mean is None:
            return x
        return (x - self.mean) / self.std

    def decode(self, x, sample_idx=None):
        if self.mean is None:
            return x
        return x * self.std + self.mean

### Model

In [ ]:
%%writefile model.py
"""Definition of the model architecture"""

import torch
import torch.nn as nn
from egnn_pytorch import EGNN


class EGNNModel(nn.Module):
    """
    EGNN-based model for stress prediction on mesh.
    """

    def __init__(self, config):
        """
        Initialize the model using parameters from the configuration.

        Args:
            config: Configuration object containing architecture and
                    EGNN hyperparameters.
        """
        super().__init__()

        # ------------------------------------------------------------------
        # Input embedding
        # Maps raw node features to the hidden dimension
        # ------------------------------------------------------------------
        self.embedding = nn.Linear(
            config.IN_NODE_FEATURES,
            config.HIDDEN_CHANNELS
        )

        # ------------------------------------------------------------------
        # Stress prediction head
        # Predicts stress values per node from hidden features
        # ------------------------------------------------------------------
        self.stress_head = nn.Sequential(
            nn.Linear(config.HIDDEN_CHANNELS, config.HIDDEN_CHANNELS),
            nn.ReLU(),
            nn.Linear(config.HIDDEN_CHANNELS, config.OUT_STRESS_DIM)
        )

        # ------------------------------------------------------------------
        # EGNN layers
        # Stack of equivariant graph neural network blocks
        # ------------------------------------------------------------------
        self.layers = nn.ModuleList([])
        for _ in range(config.DEPTH):
            self.layers.append(
                EGNN(
                    dim=config.HIDDEN_CHANNELS,              # Node feature dimension
                    m_dim=config.HIDDEN_CHANNELS,            # Message dimension
                    num_nearest_neighbors=config.NUM_NEIGHBORS,
                    update_coors=config.UPDATE_COORS,        # Whether to update coordinates
                    update_feats=config.UPDATE_FEATS,        # Whether to update features
                    norm_coors=True,                          # Normalize coordinate updates
                    soft_edges=True                           # Use soft edge weights
                )
            )

    def forward(self, h, pos, mask=None):
        """
        Forward pass of the model.

        Args:
            h (Tensor): Node features of shape (N, IN_NODE_FEATURES)
            pos (Tensor): Node coordinates of shape (N, 3)
            mask (Tensor, optional): Mask for valid nodes (for batching)

        Returns:
            Tensor: Predicted stress values per node
        """

        # Embed input node features into hidden space
        h = self.embedding(h)

        # Apply EGNN layers sequentially
        for layer in self.layers:
            h, pos = layer(h, pos, mask=mask)

        # Predict stress from final node embeddings
        pred_stress = self.stress_head(h)
        return pred_stress


def create_model(config):
    """
    Factory function to create and move the model to the correct device.

    Args:
        config: Configuration object containing device information.

    Returns:
        EGNNModel: Model placed on the specified device.
    """
    model = EGNNModel(config)
    return model.to(config.DEVICE)

### Data Loading

In [ ]:
%%writefile data_loading.py
"""Loading and preparation of the data"""

import torch
import sys
import types
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from utils import UnitGaussianNormalizer


sys.modules['utils'] = types.ModuleType('utils')
sys.modules['utils'].UnitGaussianNormalizer = UnitGaussianNormalizer


class TupleDataset(InMemoryDataset):
    """Dataset PyTorch Geometric from tuples(data, slices)."""

    def __init__(self, data, slices):
        super().__init__(None)
        self.data = data
        self.slices = slices

    def len(self):
        return self.data.num_graphs if hasattr(self.data, 'num_graphs') else self.slices['x'].size(0) - 1

    def get(self, idx):
        return self.data.__class__.from_data_list([self.data])[idx]


def get_graph_from_tuple(data, slices, idx):
    """Get's the idx_th tuple from data."""
    data_dict = {}
    for key in slices.keys():
        start, end = slices[key][idx].item(), slices[key][idx + 1].item()

        if key in ['edge_index', 'face'] and data[key].dim() == 2:
            data_dict[key] = data[key][:, start:end]
        else:
            data_dict[key] = data[key][start:end]

    #Extract the position from the features
    data_dict['pos'] = data_dict['x'][:, :3]
    data_dict['x'] = data_dict['x'][:, 3:]

    return Data(**data_dict)


def load_data(config):
    """Loads training and validation data"""
    print("="*70)
    print("DATA LOADING")
    print("="*70)

    train_dataset_tuple = torch.load(config.TRAIN_DATA_PATH, weights_only=False)
    train_data, train_slices = train_dataset_tuple

    num_graphs = train_slices['x'].size(0) - 1
    keep = int(num_graphs * config.TRAIN_SUBSET_RATIO)
    print(f"\n📊 Dataset train: {num_graphs} graphs → Kept: {keep} ({config.TRAIN_SUBSET_RATIO*100:.0f}%)")

    new_slices = {}
    for key in train_slices.keys():
        new_slices[key] = train_slices[key][:keep+1].clone()
    train_slices = new_slices

    test_dataset_tuple = torch.load(config.VAL_DATA_PATH, weights_only=False)
    test_data, test_slices = test_dataset_tuple
    print(f"📊 Dataset val: {test_slices['x'].size(0) - 1} graphs")

    # Create graohs
    print("\n🔄 Graph extraction...")
    train_graphs = [get_graph_from_tuple(train_data, train_slices, i)
                    for i in range(train_slices['x'].size(0) - 1)]
    val_graphs = [get_graph_from_tuple(test_data, test_slices, i)
                  for i in range(test_slices['x'].size(0) - 1)]

    print(f"✓ Train: {len(train_graphs)} graphs")
    print(f"✓ Val: {len(val_graphs)} graphs")

    # Créer les dataloaders
    train_loader = DataLoader(train_graphs, batch_size=config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_graphs, batch_size=config.BATCH_SIZE, shuffle=False)

    print("\n✓ DataLoaders created")
    print("="*70 + "\n")

    return train_loader, val_loader, len(train_graphs), len(val_graphs)

### Training

This script trains and validates an EGNN-based model by converting batched graphs to dense form, computing Huber loss on valid nodes, tracking per-graph R² distribution and global error metrics, saving the best model, and visualizing training dynamics over epochs.

In [ ]:
%%writefile training.py
"""Training and validation loop."""

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.utils import to_dense_batch
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import time


def train_epoch(model, train_loader, optimizer, device, gradient_clip, num_train_graphs):
    model.train()
    train_loss = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Convert sparse graph batch to dense tensors
        h_dense, mask = to_dense_batch(batch.x, batch.batch)
        pos_dense, _ = to_dense_batch(batch.pos, batch.batch)

        # Forward pass
        pred_stress_dense = model(h_dense, pos_dense, mask=mask)

        # Remove padded nodes
        pred_stress_valid = pred_stress_dense[mask].view(-1)
        target_stress = batch.y.view(-1)

        # Compute loss only on real (non-padded) nodes
        loss = F.huber_loss(pred_stress_valid, target_stress, delta=0.5)
        loss.backward()

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=gradient_clip)

        optimizer.step()
        train_loss += loss.item() * batch.num_graphs

    train_loss /= num_train_graphs
    return train_loss


def validate(model, val_loader, device, num_val_graphs):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        all_predictions_global = []
        all_targets_global = []
        r2_scores_per_graph = []

        for batch in val_loader:
            batch = batch.to(device)

            # Convert sparse graph batch to dense tensors
            h_dense, mask = to_dense_batch(batch.x, batch.batch)
            pos_dense, _ = to_dense_batch(batch.pos, batch.batch)

            # Forward pass
            pred_stress_dense = model(h_dense, pos_dense, mask=mask)

            # Remove padded nodes
            pred_stress_valid = pred_stress_dense[mask].view(-1)
            target_stress = batch.y.view(-1)

            # Compute loss only on real (non-padded) nodes
            loss = F.huber_loss(pred_stress_valid, target_stress, delta=0.5)
            val_loss += loss.item() * batch.num_graphs

            # Convert tensors to NumPy for metric computation
            preds_np = pred_stress_valid.cpu().numpy()
            targets_np = target_stress.view(-1).cpu().numpy()
            batch_indices = batch.batch.cpu().numpy()

            # Per-graph R² computation
            unique_graphs = np.unique(batch_indices)
            for graph_id in unique_graphs:
                # Select nodes belonging to the current graph
                graph_mask = (batch_indices == graph_id)

                graph_targets = targets_np[graph_mask]
                graph_preds = preds_np[graph_mask]

                if len(graph_targets) > 1:
                    score = r2_score(graph_targets, graph_preds)
                    r2_scores_per_graph.append(score)

            all_predictions_global.append(pred_stress_valid)
            all_targets_global.append(target_stress)

    val_loss /= num_val_graphs

    metrics = {}
    metrics['all_r2'] = r2_scores_per_graph

    if len(all_predictions_global) > 0:
        final_preds = torch.cat(all_predictions_global).cpu().numpy()
        final_targets = torch.cat(all_targets_global).cpu().numpy()

        metrics['RMSE'] = np.sqrt(mean_squared_error(final_targets, final_preds))
        metrics['MAE'] = mean_absolute_error(final_targets, final_preds)

        if len(r2_scores_per_graph) > 0:
            scores_array = np.array(r2_scores_per_graph)
            metrics['R2_90PCT'] = np.percentile(scores_array, 90)
            metrics['R2_50PCT'] = np.percentile(scores_array, 50)
            metrics['R2_10PCT'] = np.percentile(scores_array, 10)
        else:
            metrics['R2_90PCT'] = 0
            metrics['R2_50PCT'] = 0
            metrics['R2_10PCT'] = 0

    return val_loss, metrics


def train_model(model, train_loader, val_loader, optimizer, config, num_train_graphs, num_val_graphs):
    print("=" * 70)
    print("START OF TRAINING")
    print("=" * 70 + "\n")

    history = {
        'train_loss': [], 'val_loss': [],
        'RMSE': [], 'MAE': [],
        'R2_90PCT': [], 'R2_50PCT': [], 'R2_10PCT': []
    }

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(1, config.NUM_EPOCHS + 1):
        start_time = time.time()

        train_loss = train_epoch(
            model, train_loader, optimizer,
            config.DEVICE, config.GRADIENT_CLIP, num_train_graphs
        )
        train_losses.append(train_loss)

        val_loss, metrics = validate(
            model, val_loader, config.DEVICE, num_val_graphs
        )
        val_losses.append(val_loss)

        # Store metrics
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['RMSE'].append(metrics['RMSE'])
        history['MAE'].append(metrics['MAE'])
        history['R2_90PCT'].append(metrics['R2_90PCT'])
        history['R2_50PCT'].append(metrics['R2_50PCT'])
        history['R2_10PCT'].append(metrics['R2_10PCT'])

        print(
            f"Ep {epoch:03d} | Val: {val_loss:.4f} | RMSE: {metrics['RMSE']:.3f} | "
            f"R2(10/50/90): {metrics['R2_10PCT']:.2f} / {metrics['R2_50PCT']:.2f} | "
            f"{metrics['R2_90PCT']:.2f} | Time: {time.time() - start_time}"
        )

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), config.BEST_MODEL_PATH)

    return history, metrics


def plot_metrics(history, save_path):
    """
    Plot three figures: loss, error metrics (RMSE/MAE), and R² percentiles.
    """
    epochs = range(1, len(history['train_loss']) + 1)

    fig, axs = plt.subplots(3, 1, figsize=(10, 15), sharex=True)

    # 1. Loss curves
    axs[0].plot(epochs, history['train_loss'], label="Train Loss", color='blue')
    axs[0].plot(epochs, history['val_loss'], label="Validation Loss", color='red')
    axs[0].set_ylabel("Huber Loss")
    axs[0].set_title("Training & Validation Loss")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.6)

    # 2. RMSE & MAE
    axs[1].plot(epochs, history['RMSE'], label="RMSE", color='orange')
    axs[1].plot(epochs, history['MAE'], label="MAE", color='green')
    axs[1].set_ylabel("Error Units")
    axs[1].set_title("Global Error Metrics")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.6)

    # 3. R² percentiles (key paper metric)
    axs[2].plot(epochs, history['R2_90PCT'], label="R2 Best (90%)", linestyle='--', color='purple')
    axs[2].plot(epochs, history['R2_50PCT'], label="R2 Median (50%)", linewidth=2, color='black')
    axs[2].plot(epochs, history['R2_10PCT'], label="R2 Worst (10%)", linestyle=':', color='brown')

    # Zero line to visualize negative scores
    axs[2].axhline(y=0, color='gray', linestyle='-', alpha=0.3)

    axs[2].set_ylabel("R2 Score")
    axs[2].set_xlabel("Epochs")
    axs[2].set_title("R2 Score Distribution (Per Geometry)")
    axs[2].legend(loc='lower right')
    axs[2].grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Training curves saved to {save_path}")


def plot_losses(train_losses, val_losses, metrics, save_path):
    """
    Plot and save Huber loss curves.
    """
    best_val = min(val_losses)

    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Train Loss (Huber)", linewidth=2)
    plt.plot(val_losses, label="Validation Loss (Huber)", linewidth=2)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Loss (Huber)", fontsize=12)
    plt.title("Training & Validation Loss", fontsize=14)
    plt.grid(alpha=0.3)
    plt.legend(fontsize=11)
    plt.yscale('log')

    # Add final metrics summary
    if metrics is not None:
        r2_90 = metrics.get('R2_90PCT', None)
        r2_50 = metrics.get('R2_50PCT', None)
        r2_10 = metrics.get('R2_10PCT', None)
        rmse = metrics.get('RMSE', None)
        mae = metrics.get('MAE', None)

        def fmt(v):
            return f"{v:.4f}" if isinstance(v, (int, float, np.floating)) else "N/A"

        metrics_text = (
            f"R2_90: {fmt(r2_90)} | "
            f"R2_50: {fmt(r2_50)} | "
            f"R2_10: {fmt(r2_10)} | "
            f"RMSE: {fmt(rmse)} | "
            f"MAE: {fmt(mae)}"
        )

        plt.figtext(0.5, -0.08, metrics_text, ha='center', fontsize=10)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    print(f"✓ Loss curve saved: {save_path}")
    plt.show()

### Execution - Data loading 


This cell initializes the configuration and loads the training and validation datasets into memory.

In [ ]:
# --- CELL 1: Data loading (Run only once) ---
import importlib
import config
from data_loading import load_data

# Reload configuration to ensure the latest version is used
importlib.reload(config)
config = config.Config()

# Load training and validation data
train_loader, val_loader, num_train_graphs, num_val_graphs = load_data(config)

print("\n✅ Data loaded successfully!")

### Execution - Training and validation 

This cell initializes the model, executes the training and validation loops, computes evaluation metrics, and plots the resulting learning curves.

In [ ]:
# --- CELL 2: Training (Can be re-run) ---
import torch
from model import create_model
from training import train_model, plot_metrics  

# 2. Model creation
print("Model creation...")
model = create_model(config)

# 3. Optimizer definition
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY
)

# 4. Launch training
print("Starting training...")
# Fix unpacking: train_model returns 2 values (training history and final metrics dictionary)
history, metrics = train_model(
    model, train_loader, val_loader, optimizer, config,
    num_train_graphs, num_val_graphs
)

# Extract R² values
all_r2 = metrics['all_r2']

# 5. Visualization
# Use plot_metrics which matches the output of train_model
plot_metrics(history, config.TRAINING_CURVE_PATH)

### Qualitative Analysis: 3D Stress Field Visualization

This cell performs the final visual inspection of the model's predictions. We iterate through the specific test cases identified earlier: **Best, Median, and Worst** performance.

**Key Feature: Independent Color Scaling**
The `visualize_side_by_side_independent` function allows the color map to scale dynamically for each plot:
* **Left (Target):** Ground truth from FEM.
* **Right (Prediction):** GNN output.

**Why independent scales?**
This allows us to verify if the model has learned the correct **stress distribution patterns** (topology of the hotspots), even if the absolute **magnitudes** are underestimated (a common issue known as "over-smoothing" in regression tasks).

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
import numpy as np

best_idx = np.argmax(all_r2)
median_idx = np.argmin(np.abs(all_r2 - np.median(all_r2)))
worst_idx = np.argmin(all_r2)

val_graphs = val_loader.dataset

# --- 1. ROBUST VISUALIZATION FUNCTION ---
def visualize_results(target_data, pred_stress, original_pos, index):
    """
    Displays ground truth and EGNN prediction side by side.
    Automatically handles Mesh rendering (if faces are present) or Point Cloud rendering.
    """

    # Move data to CPU
    x = original_pos[:, 0].cpu().numpy()
    y = original_pos[:, 1].cpu().numpy()
    z = original_pos[:, 2].cpu().numpy()

    # Ground truth and prediction
    val_target = target_data.y.view(-1).cpu().numpy()
    val_pred = pred_stress.view(-1).cpu().numpy()

    # Common color scale
    global_min = min(val_target.min(), val_pred.min())
    global_max = max(val_target.max(), val_pred.max())

    target_min, target_max = val_target.min(), val_target.max()
    pred_min, pred_max = val_pred.min(), val_pred.max()

    # Check: do we have faces for mesh rendering?
    has_faces = hasattr(target_data, 'face') and target_data.face is not None

    if has_faces:
        faces = target_data.face.cpu().numpy()
        i, j, k = faces[0], faces[1], faces[2]
        trace_type = go.Mesh3d
        kwargs = dict(i=i, j=j, k=k, intensitymode='vertex')
    else:
        print("ℹ️ No faces detected, rendering as point cloud (Scatter3d).")
        trace_type = go.Scatter3d
        kwargs = dict(mode='markers', marker=dict(size=4))

    # --- Figure creation ---
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=(f"Target FEM (Graph {index})", f"EGNN Prediction (Graph {index})")
    )

    # Trace 1: Ground truth
    fig.add_trace(
        trace_type(
            x=x, y=y, z=z,
            intensity=val_target,
            colorscale='Jet',
            cmin=target_min, cmax=target_max,
            opacity=1.0,
            name='Target',
            colorbar=dict(
                title="Target Stress",
                x=0.45,       # Positioned between the two plots
                len=0.7,      # Colorbar length
                thickness=15
            ),
            **kwargs
        ),
        row=1, col=1
    )

    # Trace 2: Prediction
    fig.add_trace(
        trace_type(
            x=x, y=y, z=z,
            intensity=val_pred,
            colorscale='Jet',
            cmin=pred_min, cmax=pred_max,
            opacity=1.0,
            name='Prediction',
            colorbar=dict(
                title="Predicted Stress",
                x=1.0,        # Positioned on the far right
                len=0.7,
                thickness=15
            ),
            showscale=True,
            **kwargs
        ),
        row=1, col=2
    )

    fig.update_layout(
        title_text=f"Von Mises Comparison – Independent Scales (Mesh {index})",
        height=600,
        width=1250,  # Slightly wider to accommodate colorbars
        margin=dict(r=20, b=0, l=0, t=50),
        scene=dict(aspectmode='data'),
        scene2=dict(aspectmode='data')
    )

    fig.show()

# --- 2. EXECUTION ---
# Choose a valid index (make sure it is < len(val_graphs))
INDEX = best_idx
print(all_r2[INDEX])

print(f"--- Preparing visualization for sample #{INDEX} ---")

# 1. Retrieve the graph
target_graph = val_graphs[INDEX]

# 2. Save original positions (CPU) for Plotly
pos_orig = target_graph.pos.clone()

# 3. Prepare data for the EGNN model
# The model expects: [Batch, Nodes, Features], so we add a batch dimension (unsqueeze)
h_input = target_graph.x.unsqueeze(0).to(config.DEVICE)     # Shape: [1, N, Features]
pos_input = target_graph.pos.unsqueeze(0).to(config.DEVICE) # Shape: [1, N, 3]

# Create a mask (all nodes are valid here, no padding)
mask_input = torch.ones(
    (1, target_graph.x.shape[0]),
    dtype=torch.bool
).to(config.DEVICE)

# 4. Prediction
model.eval()
with torch.no_grad():
    # Correct call according to your EGNNModel(h, pos, mask) definition
    pred_dense = model(h_input, pos_input, mask=mask_input)

    # Remove batch dimension and move back to CPU
    pred_stress = pred_dense.squeeze(0).view(-1).cpu()

# 5. Visualization
visualize_results(target_graph, pred_stress, pos_orig, INDEX)